# [Advent of Code 2020 Day 16](https://adventofcode.com/2020/day/16)

Deep diving into CSPs.

## Initial setup

In [ ]:
import ipytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()

## Input Parsing

In [ ]:
def parse_input(filename: str) -> tuple[tuple[str, str, str, str, str], tuple[int, ...], tuple[tuple[int, ...], ...]]:

    gen = yield_line(filename)

    rules = []

    while (next_line := next(gen)) != "":
        rules.append(parse(f"(.+): (\d+)\-(\d+)\ or (\d+)\-(\d+)", next_line))

    assert next(gen).startswith("your ticket:")

    your_ticket = intsep(next(gen), ",")

    assert next(gen) == ""
    assert next(gen).startswith("nearby tickets:")

    other_tickets = []

    for line in gen:
        other_tickets.append(tuple(intsep(line, ",")))

    return tuple(rules), tuple(your_ticket), tuple(other_tickets)

## Part 1
This seems like one of those codebreaker-type problems requiring a lot of logic (and probably set theory). I'm going to start off by making rule validation functions.

### Rule Class
yum OOP

In [ ]:
class Rule:

    def __init__(self, name: str, min_a: int, max_a: int, min_b: int, max_b: int):
        assert min_a < max_b, f"Ranges must be left-to-right - args: {name=} {min_a=} {max_a=} {min_b=} {max_b=}"
        assert min_a <= max_a, f"First range must be left-to-right - args: {name=} {min_a=} {max_a=} {min_b=} {max_b=}"
        assert min_b <= max_b, f"Second range must be left-to-right - args: {name=} {min_a=} {max_a=} {min_b=} {max_b=}"
        # assert max_a < min_b, f"There must be no overlap - args : {name=} {min_a=} {max_a=} {min_b=} {max_b=}"
        self.name: str = name
        self.bound_a: tuple[int, int] = (min_a, max_a)
        self.bound_b: tuple[int, int] = (min_b, max_b)
        self.cardinality = (max_a - min_a + 1) + (max_b - min_b + 1) if max_a < min_b else max_b - min_a + 1

    def validate(self, value: int):
        return self.bound_a[0] <= value <= self.bound_a[1] or self.bound_b[0] <= value <= self.bound_b[1]

    def __str__(self):
        return f"{self.name}: {self.bound_a[0]}-{self.bound_a[1]} or {self.bound_b[0]}-{self.bound_b[1]}"

    def __repr__(self):
        return str(self)

In [ ]:
%%ipytest
def test_rule_class_class():
    class_rule: Rule = Rule("class", 0, 1, 4, 19)
    assert class_rule.cardinality == 18
    for illegal in [-1, 2, 3]:
        assert not class_rule.validate(illegal)
    for legal in [0, 1, 4, 19, 11]:
        assert class_rule.validate(legal)

def test_rule_class_overlapping_abut():
    overlapping_rule: Rule = Rule("overlapping", 0, 5, 5, 20)
    assert overlapping_rule.cardinality == 21
    for illegal in [-1, -100, 21, 200]:
        assert not overlapping_rule.validate(illegal)
    for legal in [0, 20, 10, 5, 3, 4, 6, 2, 4]:
        assert overlapping_rule.validate(legal)

def test_rule_class_overlapping_past():
    overlapping_rule: Rule = Rule("overlapping", 0, 5, 3, 20)
    assert overlapping_rule.cardinality == 21
    for illegal in [-1, -100, 21, 200]:
        assert not overlapping_rule.validate(illegal)
    for legal in [0, 20, 10, 5, 3, 4, 6, 2, 4]:
        assert overlapping_rule.validate(legal)

### Now to make filtering functions

In [ ]:
def value_passes_any_rule(rules: list[Rule], candidate_value: int):
    for rule in rules:
        if rule.validate(candidate_value):
            return True
    return False

def get_first_unsatisfiable_value_on_ticket(rules: list[Rule], ticket: list[int]) -> int | None:
    for num in ticket:
        if not value_passes_any_rule(rules, num):
            return num
    return None

Too lazy to test.

In [ ]:
def part_one(data: Any) -> int | str:
    rules_input, your_ticket, other_tickets = data

    rules = []

    for name, l1, l2, r1, r2 in rules_input:
        rules.append(Rule(name, int(l1), int(l2), int(r1), int(r2)))

    result = 0

    for ticket in other_tickets:
        if (unsatisfiable_value := get_first_unsatisfiable_value_on_ticket(rules, ticket)) is not None:
            result += unsatisfiable_value

    return result

In [ ]:
%%ipytest
def test_part_one():
    assert part_one(parse_input("example1")) == 71
    assert part_one(parse_input("input")) == 27850

## Part 2
OK, so this is where the logic comes in... this looks like a substitution cipher type problem on LeetCode (think "isomorphic strings" with a one-to-one mapping between symbols).

The question, in true CSP fashion, asks for an assignment of which field goes where. The hashing function used by this question (aka to determine the correct answer), assuming the ordering is correct, is to take every field that starts with "departure" and multiply them together. Therefore, I will probably solve the CSP problem first, then create the response converter.

### Transform-and-Conquer
One important thing to notice is that we can make some optimizations based off the fact that columns are static (i.e., if ticket #1 column 1 is "height", then every other ticket's column 1 is "height" too).

#### Hashing
The thing stopping us from doing a pure interval reduction is the fact that rules have two intervals, unfortunately. But I think we can still use interval reduction as a means of smoke testing, similar to how hashing works:
- Take every column and reduce it to an interval based on the min and max value
  - For example, the column `[4, 5, 9, 1, 15, 14, 13]` would be of the interval `[1, 15]`
- Use this interval as a hash for the column
  - For the column above, we can be certain for a fact that it will fail the rule `height: 0-1 or 3-4`
  - However, for the same column, we can't make an immediate deduction on the rule `height: 0-1 or 14-15` because even though the column matches the interval, this property is **necessary, but not sufficient**. Therefore, we would have to continue to manually test it. We would find that the column does fail, because the value 4 isn't allowed.
    - You will see this "smoke test" line of logic in the pumping lemma, Miller-Rabin's primality testing, and using degree sequence to screen out isomorphism between graphs

#### Proof by Contradiction
This is a form of transform-and-conquer known as "problem reduction" (see Levitin, A.'s algorithm book for more info.) where you solve a problem using an equivalent problem (for example, in high school you use problem reduction when finding known extrema of polynomials by instead solving for its derivative - you are solving for min/max by instead solving for 0).

We can do the same here. Take a look at this statement:
$$\textrm{Every number in column $c$ satisfies rule $r$'s two intervals} \Rightarrow \textrm{$c$ satisfies $r$}$$

However, by taking the contrapositive, we can form an equally-consistent statement:
$$\textrm{There exists a single number $n$ in column $c$ that does not satisfy rule $r$'s two intervals} \Rightarrow \textrm{$c$ does not satisfy $r$}$$

It doesn't matter if the function checks if a column satisfies a rule or not - if I want the other, I can just invert it! So long as these functions are consistent and complete. This is actually how Python implements some relational operators (for example, if you override "equals", you don't have to override "not equals" because you can just invert the "equals" override):
> By default, `__ne__()` delegates to `__eq__()` and inverts the result unless it is `NotImplemented`. There are no other implied relationships among the comparison operators, for example, the truth of (`x<y` or `x==y`) does not imply `x<=y`.

This means, instead of checking if a large number of integers fit within a given rule, we can instead look at a rule's disallowed integers and see if they exist within a given column. Looking at the full test input we see:
```
departure location: 47-691 or 713-954
departure station: 44-776 or 799-969
departure platform: 37-603 or 627-953
departure track: 41-240 or 259-955
departure date: 42-370 or 383-961
departure time: 50-117 or 136-962
arrival location: 33-86 or 104-973
arrival station: 29-339 or 347-962
arrival platform: 46-644 or 659-970
arrival track: 31-584 or 604-960
class: 42-107 or 115-971
duration: 31-753 or 770-972
price: 40-515 or 525-957
route: 31-453 or 465-971
row: 46-845 or 868-965
seat: 45-475 or 489-960
train: 34-317 or 323-968
type: 47-150 or 159-969
wagon: 45-261 or 279-955
zone: 33-879 or 891-952
```

This is particularly a good idea because we don't actually have to check numbers below the left bound's min or the right bound's max because we already precomputed each column's range before reaching this step. This means a column with the range `[5, 952]` will never actually be checked against rules like `zone: 33-879 or 891-952` because the 5 in the range automatically eliminates itself (as the absolute minimum for rule `zone` is 33).

Columns `[33, 880, 952]` and `[33, 34, 952]` both have the same range `[33, 952]`, and would move forward to validation. However, rather than checking every number in each column to see if it fits with the rule, we can just check the rejection range for the rule, which in this case is 880 through 890 - we only have to check 10 numbers, and if we represent each column as a hashset, we can check each potential rejections' existences in $\mathcal{O}(1)$ amortized time. First, let's create the column cruncher function.

In [ ]:
class Column:

    INFINITY = 0x3f3f3f3f

    def __init__(self) -> None:
        """
        Initialize the Column.
        """
        self.sum: int = 0
        self.set: set[int] = set()
        self.min: int = Column.INFINITY
        self.max: int = -Column.INFINITY

    def __iadd__(self, other: int) -> "Column":
        """
        Add an integer to the column's bookkeeping:
            - Add to total sum
            - Add to set for O(1) membership checking
            - Update max and min
        :param other: the integer to add to the column
        :return the current instance
        """
        # assert other not in self.set, "Debug - are there duplicates?" # EDIT yes there are
        self.sum += other
        self.set.add(other)
        self.min = min(self.min, other)
        self.max = max(self.max, other)
        return self

    def dump(self) -> tuple[int, set[int], int, int]:
        """
        Dumps the four fields of the column into array (for easy testing)
        :return: a tuple representing the data of the class
        """
        return self.sum, self.set, self.min, self.max

    def __str__(self) -> str:
        return f"Column(sum={self.sum}, set={self.set}, min={self.min}, max={self.max})"

    def __repr__(self) -> str:
        return str(self)

In [ ]:
%%ipytest
def test_column_class_simple():
    column = Column()
    assert column.dump() == (0, set(), Column.INFINITY, -Column.INFINITY)
    test_steps = [
        (0, 0, {0}, 0, 0),
        (5, 5, {0, 5}, 0, 5),
        (-10, -5, {0, 5, -10}, -10, 5),
        (6, 1, {0, 5, -10, 6}, -10, 6),
    ]
    for next_input, expected_sum, expected_set, expected_min, expected_max in test_steps:
        column += next_input
        assert column.dump() == (expected_sum, expected_set, expected_min, expected_max)

def test_column_class_sum():
    column = Column()
    nums = [235,447,575,80,384,832,799,806,529,624,144,398,176,583,199,169,914,222,828,314]
    for num in nums:
        column += num
    assert column.sum == sum(nums)

### CSPs, assignments, consistency, oh my!
In simple terms, a constraint satisfaction problem consists of a set of variables; each variable has a domain of values it can take on. The CSP has a series of constraints that the variables must follow. The aim of solving a CSP involves creating some assignment of values in each variable's domain such that all the constraints are followed.

For example, if you are looking for four unique integers that add up to 10, then you have four variables, $integer_1$, $integer_2$, $integer_3$, and $integer_4$; the domain of each of these variables is $\{0, 1, 2, 3, 4, 5, 6, 7, 8, 9\}$, and the constraints of the problem are that:
- $integer_1 + integer_2 + integer_3 + integer_4 = 10$
- $ALLDIFF(integer_1, integer_2, integer_3, integer_4)$
where $ALLDIFF$ (or $alldiff$ or $alldifferent$) refers to the constraint that all integers must be distinct.

One assignment to this CSP that solves it is $\{1, 2, 3, 4\}$.

This seems like a very boring explanation, but this is the iceberg to a very big part of AI. Most LeetCode problems are either CSP or search problems. CSPs form the first step beyond atomic states in Norvig's AI data representation models (search, for an example, has to do with atomic states); CSPs are part of the "factored" representation where we view states as vectors of different variables.

Most CSP solvers do something called "backtracking search" where you incrementally build up a candidate solution. How a CSP solver performs this varies. Massively. We will see in a bit.

### Solution Verification Wrapper
If our generic backtracker is going to return a list of integers, one for each column, each integer representing which field/rule, then we want to only check certain indices. In the original problem, only the fields that start with "departure". This means if we look at our test input:
```
departure location: 47-691 or 713-954
departure station: 44-776 or 799-969
departure platform: 37-603 or 627-953
departure track: 41-240 or 259-955
departure date: 42-370 or 383-961
departure time: 50-117 or 136-962
arrival location: 33-86 or 104-973
arrival station: 29-339 or 347-962
arrival platform: 46-644 or 659-970
arrival track: 31-584 or 604-960
class: 42-107 or 115-971
duration: 31-753 or 770-972
price: 40-515 or 525-957
route: 31-453 or 465-971
row: 46-845 or 868-965
seat: 45-475 or 489-960
train: 34-317 or 323-968
type: 47-150 or 159-969
wagon: 45-261 or 279-955
zone: 33-879 or 891-952
```
this corresponds to indices `[0, 1, 2, 3, 4, 5]`. CSPs rarely have only one result, so just checking the elements one-by-one would be a very fragile way of testing it. Let's make something more robust.

In [ ]:
def digest_ticket_using_indices(ticket: tuple[int, ...], assignment: tuple[int, ...], desired_rules: tuple[int, ...]) -> int:
    """
    Takes a ticket consisting of a bunch of integers and an array of indices, then returns a digest consisting of every position in ticket in indices multiplied together.
    :param ticket: a list of integers representing the numbers taken on a ticket
    :param assignment: a list of integers signifying which rule is assigned to each index of the ticket
    :param desired_rules: a list of n integers, n <= len(ticket), representing what parts of the ticket should be multiplied together to form the digest
    :return: an integer digest
    """

    assert len(ticket) == len(assignment), f"The number of entries on a ticket should be equal to the number of assignments. {ticket=} {assignment=}"
    assert set(assignment) >= set(desired_rules), "Desired rules {desired_rules} must contain only rules tht exist in the assignment {assignment}"
    assert len(set(ticket)) == len(ticket), "Ticket {ticket} must not have duplicate numbers"

    locations: dict[int, int] = {}

    for idx, rule_num in enumerate(assignment):
        locations[rule_num] = idx

    result: int = 1

    for rule_index in desired_rules:
        result *= ticket[locations[rule_index]]

    return result

In [ ]:
%%ipytest
def test_digest_ticket_using_indices():

    assert digest_ticket_using_indices(
        (191, 139, 59, 79, 149, 83, 67, 73, 167, 181, 173, 61, 53, 137, 71, 163, 179, 193, 107, 197),
        (16, 8, 3, 4, 10, 9, 18, 5, 19, 12, 1, 2, 15, 0, 13, 17, 7, 11,  6, 14),
        (0, 1, 2, 3, 4, 5)
    ) == 491924517533

    assert digest_ticket_using_indices(
        (191, 139, 59, 79, 149, 83, 67, 73, 167, 181, 173, 61, 53, 137, 71, 163, 179, 193, 107, 197),
        (16, 8, 3, 4, 10, 9, 18, 5,  6, 12, 1, 2, 15, 0, 13, 17, 7, 11, 19, 14),
        (0, 1, 2, 3, 4, 5)
    ) == 491924517533

    assert digest_ticket_using_indices(
        (191, 139, 59, 79, 149, 83, 67, 73, 167, 181, 173, 61, 53, 137, 71, 163, 179, 193, 107, 197),
        (16, 8, 3, 4, 10, 9, 18, 5, 13, 12, 1, 2, 15, 0, 19, 17, 7, 11,  6, 14),
        (0, 1, 2, 3, 4, 5)
    ) == 491924517533

### Brute Force
Yep, you already know it. I love brute force algorithms so much... though I wouldn't say this is the most naive brute force algorithm, which would be to literally generate all $n!$ permutations. Rather, at every step, it chooses every consistent assignment for the next immediate variable (represented by the variable `idx`). Most of the cose is setting stuff up and pre-computation, but the actual computation time lies squarely in the `csp_backtrack` function which actually branches out from the possible assignments.

Essentially, the skeleton of a CSP backtracker is this:

__function__ SolveCSP(_assignment_);<br/>
    _assignment_ $\leftarrow$ an incomplete CSP assignment<br/>
    <br/>
    **if** _assignment is complete_ **then**<br/>
        **return** _assignment_<br/>
    **end if**<br/>
    <br/>
    _variable_ $\leftarrow$ GetNextUnassignedVariable(_assignment_)<br/>
    **for each** _value_ $\in$ GetValues(_variable_) **do**<br/>
        _assignment_ $\leftarrow$ AssignValue(_variable_, _value_)<br/>
        **if** SolveCSP(_assignment_) $\ne$ _FAILURE_ **then**<br/>
            **return** _assignment_<br/>
        **end if**<br/>
        _assignment_ $\leftarrow$ UnassignValue(_variable_, _value_)<br/>
    **end for**<br/>
    <br/>
    **return** _FAILURE_<br/>

In [ ]:
def infer_fields_based_on_ticket_values_brute_force(rule_tuples: tuple[tuple[str, str, str, str, str]], your_ticket: tuple[int, ...], other_tickets: tuple[tuple[int, ...]]) -> tuple[int, ...]:
    """
    Takes a bunch of rules, each containing two non-overlapping, allowed intervals, and several tickets, removes the unsatisfiable ones, and then attempts to infer which field is which
    :param rule_tuples: tuple of tuples, each representing a rule; contains a name, then four integers; each pair of two integers represents an interval
    :param your_ticket: your ticket; not used specifically in this function, but is treated as any other ticket for the sake of logical deduction
    :param other_tickets: the other tickets, each containing a number of integers potentially satisfiable w.r.t. the rule tuples
    :return: a tuple consisting of n integers, n being the number of columns, with each integer i representing an assignment of the i'th rule
    """

    # More guardrails
    assert len(set([len(ticket) for ticket in other_tickets])) == 1, "Tickets must all have the same amount of columns"
    assert len(your_ticket) == len(other_tickets[0]), "Your ticket must have the same number of columns as the others too"
    assert len(rule_tuples) == len(your_ticket), "There must be an equal number of rules to columns"

    # Number of columns and number of rules
    n: int = len(your_ticket)

    # Initialize rules using the data in the tuple
    rules: list[Rule] = [Rule(name, int(mn_1), int(mx_1), int(mn_2), int(mx_2)) for name, mn_1, mx_1, mn_2, mx_2 in rule_tuples]

    # Initialize columns to 0
    columns: list[Column] = [Column() for _ in range(n)]

    # Gather your ticket with the others for filtering
    all_tickets_unclean: tuple[tuple[int, ...], ...] = (*other_tickets, your_ticket)

    # Filter the tickets
    all_tickets: list[tuple[int, ...], ...] = [ticket for ticket in all_tickets_unclean if get_first_unsatisfiable_value_on_ticket(rules, ticket) is None]

    # Populate columns
    for ticket in all_tickets:
        for col_idx, col_val in enumerate(ticket):
            columns[col_idx] += col_val

    # Checking if the assignment of rule to column is consistent
    @lru_cache(None)
    def consistent_assignment(column: Column, rule: Rule) -> bool:

        # If there exists a single value outside the external boundaries return false
        if column.min < rule.bound_a[0] or column.max > rule.bound_b[1]:
            return False

        # If the column is completely contained within one of the rules' intervals return true
        if (column.min >= rule.bound_a[0] and column.max <= rule.bound_a[1]) or (column.min >= rule.bound_b[0] and column.max <= rule.bound_b[1]):
            return True

        # If the column is completely contained within the rule's danger zone
        if column.min > rule.bound_a[1] and column.max < rule.bound_b[0]:
            return False

        # Check if the rejection numbers are present in the column
        rejection_interval_min, rejection_interval_max = rule.bound_a[1] + 1, rule.bound_b[0] - 1
        for potential_rejection in range(rejection_interval_min, rejection_interval_max + 1):
            if potential_rejection in column.set:
                return False

        # Guilty until proven innocent
        return True

    @lru_cache(None)
    def get_column_agreeable_ranking(column_idx: int):
        count = 0
        for rule_idx in range(n):
            if consistent_assignment(columns[column_idx], rules[rule_idx]):
                count += 1
        return count

    # Build up a candidate solution
    def csp_backtrack(columns_left: list[int], rules_left: set[int], curr_assignment: list[int]) -> bool:

        if len(columns_left) == 0:
            return True

        column_idx: int = columns_left.pop()

        for rule_idx in rules_left:
            if consistent_assignment(columns[column_idx], rules[rule_idx]):
                curr_assignment[column_idx] = rule_idx
                rules_left.remove(rule_idx)
                if csp_backtrack(columns_left, rules_left, curr_assignment):
                    return True
                rules_left.add(rule_idx)
                curr_assignment[column_idx] = -1

        columns_left.append(column_idx)

        return False

    new_assignment = [-1] * n
    columns_left = sorted(list(range(n)), key=lambda column_idx: get_column_agreeable_ranking(column_idx), reverse=True)
    rules_left = set(range(n))

    assert csp_backtrack(columns_left, rules_left, new_assignment), "Function should always find a valid solution"

    for i in range(len(new_assignment)):
        assert consistent_assignment(columns[i], rules[new_assignment[i]]), f"column_idx {i} not consistent with rule_idx {new_assignment[i]}"

    print(f"Returning valid assignment {new_assignment}")

    return tuple(new_assignment)

In [ ]:
%%ipytest -x
import pytest

def test_bruteforce_csp_example():
    example_input = parse_input("example2")
    rules_input: tuple[tuple[str, str, str, str, str]] = example_input[0]
    your_ticket: tuple[int, ...] = example_input[1]
    other_tickets: tuple[tuple[int, ...], ...] = example_input[2]

    ticket_assignment: tuple[int, ...] = infer_fields_based_on_ticket_values_brute_force(rules_input, your_ticket, other_tickets)

    assert digest_ticket_using_indices(your_ticket, ticket_assignment, (1, 0)) == 132
    assert digest_ticket_using_indices(your_ticket, ticket_assignment, (1, 2)) == 143
    assert digest_ticket_using_indices(your_ticket, ticket_assignment, (0, 2)) == 156
    assert digest_ticket_using_indices(your_ticket, ticket_assignment, (0, 1)) == 132
    assert digest_ticket_using_indices(your_ticket, ticket_assignment, (2, 1)) == 143
    assert digest_ticket_using_indices(your_ticket, ticket_assignment, (2, 0)) == 156

@pytest.mark.parametrize("filename, result", [
    ("input", 491924517533), ("input2", 239727793813), ("input3", 3902565915559), ("input4", 1382443095281), ("input5", 279139880759), ("input6", 1515506256421), ("input7", 517827547723)
])
def test_bruteforce_csp_big(filename, result):
    print(f"{filename=} expecting {result}")
    example_input = parse_input(filename)
    rules_input: tuple[tuple[str, str, str, str, str]] = example_input[0]
    your_ticket: tuple[int, ...] = example_input[1]
    other_tickets: tuple[tuple[int, ...], ...] = example_input[2]
    ticket_assignment: tuple[int, ...] = infer_fields_based_on_ticket_values_brute_force(rules_input, your_ticket, other_tickets)
    assert digest_ticket_using_indices(your_ticket, ticket_assignment, (0, 1, 2, 3, 4, 5)) == result

It takes about 15 minutes to solve it on my beefy PC. I'm surprised it worked, honestly. But there are a lot of optimizations to make. I had the right idea with making a non-trivial "consistency" function, but I feel like we can go further than that.